# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import csv
import datetime
import time
from citipy import citipy
import random
from scipy.stats import linregress

# Google API Key
from api_keys import g_key

## Generate Cities List

In [2]:
# Setup the ranges for latitude and longitude
latitude_range = (-90,90)
longatude_range = (-180,180)

# Create variables for data
cities = []

no_cities = 2000
for cntr in range(no_cities):
    latitude = random.uniform(latitude_range[0],latitude_range[1])
    longitude = random.uniform(longatude_range[0],longatude_range[1])
    
    # If we have chosen a latitude of 0 then we don't want to use that
    # one
    if latitude != 0:
        city = citipy.nearest_city(latitude,longitude)
        city_name = city.city_name
        city_country = city.country_code
        if not any(c['name'] == city_name and c['country'] == city_country for c in cities):
            city_dict = {'name': city_name,'country': city_country}
            cities.append(city_dict)

In [3]:
print(cities)

[{'name': 'hilo', 'country': 'us'}, {'name': 'sambava', 'country': 'mg'}, {'name': 'san andres', 'country': 'co'}, {'name': 'faya', 'country': 'td'}, {'name': 'pevek', 'country': 'ru'}, {'name': 'new norfolk', 'country': 'au'}, {'name': 'kushima', 'country': 'jp'}, {'name': 'tokur', 'country': 'ru'}, {'name': 'rikitea', 'country': 'pf'}, {'name': 'kahului', 'country': 'us'}, {'name': 'taolanaro', 'country': 'mg'}, {'name': 'atuona', 'country': 'pf'}, {'name': 'vao', 'country': 'nc'}, {'name': 'mataura', 'country': 'pf'}, {'name': 'silifke', 'country': 'tr'}, {'name': 'la roda', 'country': 'es'}, {'name': 'polson', 'country': 'us'}, {'name': 'gat', 'country': 'ly'}, {'name': 'skelleftea', 'country': 'se'}, {'name': 'hobart', 'country': 'au'}, {'name': 'cabo san lucas', 'country': 'mx'}, {'name': 'avarua', 'country': 'ck'}, {'name': 'punta arenas', 'country': 'cl'}, {'name': 'te anau', 'country': 'nz'}, {'name': 'bartica', 'country': 'gy'}, {'name': 'saskylakh', 'country': 'ru'}, {'name'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city_id = []
city_name = []
city_cloudiness = []
city_country = []
city_date = []
city_humidity = []
city_latitude = []
city_longitude = []
city_temperature = []
city_wind_speed = []


# Build partial query URL
query_url = f"{base_url}appid={g_key}&units={units}&q="

city_cntr = -1
no_cities_to_retrieve = 50


# Loop through cities
for index, city in enumerate(cities):
    # See if we need to sleep so we don't go over our 60 a minute max for the api
    q, mod = divmod(index+1,55)
    if mod == 0:
        print("Pausing for a minute so we don't over the api calls per minute limit!")
        time.sleep(60)
    
    
    
    # Retrieve data information
    try:
        city_json = requests.get(query_url + city['name'] + ', ' + city['country']).json()
    except:
        print(f"City not found - {city['name']}, {city['country']} - skipping")

    if city_json['cod'] == 200:
        city_cntr += 1
        print(f"Retrieved data for {city_cntr}-{city_json['name']}, {city_json['sys']['country']} ({city_json['coord']['lat']},{city_json['coord']['lon']})")
        
        city_id.append(city_cntr)
        city_name.append(city_json['name'])
        city_cloudiness.append(city_json['clouds']['all'])
        city_country.append(city_json['sys']['country'])
        city_date.append(city_json['dt'])
        city_humidity.append(city_json['main']['humidity'])
        city_latitude.append(city_json['coord']['lat'])
        city_longitude.append(city_json['coord']['lon'])
        city_temperature.append(city_json['main']['temp_max'])              
        city_wind_speed.append(city_json['wind']['speed'])
    
    if city_cntr+1 == no_cities_to_retrieve:
        break
    
          

Retrieved data for 0-Hilo, US (19.73,-155.09)
Retrieved data for 1-Sambava, MG (-14.27,50.17)
Retrieved data for 2-San Andrés, CO (12.58,-81.7)
Retrieved data for 3-Pevek, RU (69.7,170.31)
Retrieved data for 4-New Norfolk, AU (-42.78,147.06)
Retrieved data for 5-Kushima, JP (31.46,131.23)
Retrieved data for 6-Tokur, RU (53.13,132.9)
Retrieved data for 7-Rikitea, PF (-23.12,-134.97)
Retrieved data for 8-Kahului, US (20.89,-156.47)
Retrieved data for 9-Atuona, PF (-9.8,-139.03)
Retrieved data for 10-Vao, NC (-22.67,167.48)
Retrieved data for 11-Silifke, TR (36.38,33.93)
Retrieved data for 12-La Roda, ES (39.22,-2.15)
Retrieved data for 13-Polson, US (47.69,-114.16)
Retrieved data for 14-Skellefteå, SE (64.75,20.95)
Retrieved data for 15-Hobart, AU (-42.88,147.33)
Retrieved data for 16-Cabo San Lucas, MX (22.89,-109.91)
Retrieved data for 17-Avarua, CK (-21.21,-159.78)
Retrieved data for 18-Punta Arenas, CL (-53.15,-70.92)
Retrieved data for 19-Te Anau, NZ (-45.42,167.72)
Retrieved data f

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# Create a DataFrame with the following columns:
# City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
city_data = {
             "City ID": city_id,
             "City": city_name,
             "Cloudiness": city_cloudiness,
             "Country": city_country,    
             "Date": city_date,
             "Humidity": city_humidity,
             "Lat": city_latitude,
             "Lng": city_longitude,
             "Max Temp": city_temperature,
             "Wind Speed": city_wind_speed
            }

city_df = pd.DataFrame(city_data)
city_df.set_index('City ID', inplace = True)

# Write the cities.csv file
csv_file_name = "../output_data/cities.csv"

city_df.to_csv(csv_file_name)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
# Get rid of any cities that have a humitidy greater than 100
cleaned_city_df = city_df[city_df['Humidity'] <= 100]

# Preview the cleaned data
cleaned_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Hilo,75,US,1603043179,69,19.73,-155.09,73.40,5.82
1,Sambava,0,MG,1603043177,79,-14.27,50.17,73.90,11.90
2,San Andrés,75,CO,1603043301,74,12.58,-81.70,87.80,11.41
3,Pevek,30,RU,1603043178,94,69.70,170.31,23.54,16.64
4,New Norfolk,75,AU,1603043412,76,-42.78,147.06,48.20,6.93
5,Kushima,76,JP,1603043413,87,31.46,131.23,59.00,1.99
6,Tokur,0,RU,1603043413,93,53.13,132.90,8.08,3.15
7,Rikitea,68,PF,1603043413,79,-23.12,-134.97,74.89,21.18
8,Kahului,20,US,1603043179,78,20.89,-156.47,77.00,3.36


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [7]:
# Create a variable to store today's date
today = datetime.datetime.now().strftime('%m/%d/%Y')

In [10]:
def city_plot(df,compare_what,plot_label,y_label):
    # Setup the x and y axis
    x_axis = df['Lat']
    y_axis = df[compare_what]

    # Create the scatter plot
    plt.scatter(x_axis, y_axis)

    # Setup the title and labels
    plt.title(f'City Latitude vs. {plot_label} ({today})', fontsize=12)
    plt.xlabel('Latitude', fontsize=12)
    plt.ylabel(f'{y_label}', fontsize=12)
    plt.grid(True)

    # Show the scatter plot
    plt.show()    

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression